# Iterative optimisation

In [1]:
import numpy as np

<img src='img/iter_opt.png' style='height:300px;width:auto;'/>

In [83]:
def euc(x, y):
    return np.linalg.norm(x - y)

def classify(x, centers):
    dists = []
    for c in centers:
        dists.append(euc(x, c))
    return centers[dists.index(min(dists))]

def iterative_optimisation(X, clusters, max_iter=100, threshold = 0.1):
    changed = True
    cost = np.Infinity
    iteration = 0 
    while changed == True and iteration < max_iter:
        for i, x in enumerate(X):
            iteration += 1
            print('-----------------------------')
            print('ITERATION:', iteration)
            x = list(x)
            print('point:', x)
            centers = clusters.keys()
            
            dists = []
            for cluster in clusters.keys():
                dists.append(euc(np.array(list(cluster)), x))
            center = list(centers)[dists.index(min(dists))]
            print('cluster:', center)
            
            for c, points in clusters.items():
                if x in points:
                    clusters[c].remove(x)
            clusters[center].append(x)
            #clusters[center].append(x)
            if len(clusters[center]) > 1:
                costs = []
                for cluster, points in clusters.items():
                    if cluster == center:
                        costs.append((len(cluster)/(len(cluster)-1))*euc(x, np.array(center)))
                    else:
                        costs.append((len(cluster)/(len(cluster)+1))*euc(x, np.array(center)))
                k = costs.index(min(costs))
                print('cheapest cluster:', list(clusters.keys())[k])
                # if a cheaper configuration is present, move the point to the other cluster
                if list(clusters.keys())[k] != center:
                    clusters[center].remove(x)
                    clusters[list(clusters.keys())[k]].append(x)
            costs = []
            # recompute centroids
            new_clusters = {}
            for key, points in clusters.items():
                new_centroid = list(np.mean(np.array(points), axis=0))
                if (new_centroid[0], new_centroid[1]) != key:
                    #clusters.pop(key)
                    new_clusters[(new_centroid[0], new_centroid[1])] = points
                    #append cost
                    dists = [euc(np.array(p), np.array(new_centroid)) for p in points]
                    costs.append(sum(dists))
                else:
                    new_clusters[(new_centroid[0], new_centroid[1])] = points
                    dists = [euc(np.array(p), np.array(new_centroid)) for p in points]
                    costs.append(sum(dists))
            clusters = new_clusters
            # recompute cost
            new_cost = sum(costs)
            if abs(cost - new_cost) < threshold:
                changed = False
                break
            cost = new_cost
            print('new clusters:', clusters)
            print('cost:', cost)

## Examples

In [84]:
X = np.array([
    [-1, 3],
    [1, 4],
    [0, 5],
    [4, -1],
    [3, 0],
    [5, 1]
])

clusters = {
    (2.3, 1.34) : [[-1, 3], [3, 0], [5, 1]], 
    (0., 2.7) : [[1, 4], [0, 5], [4, -1]]
}

In [86]:
iterative_optimisation(X, clusters, max_iter=100, threshold=0.01)

-----------------------------
ITERATION: 1
point: [-1, 3]
cluster: (0.0, 2.7)
cheapest cluster: (2.3, 1.34)
new clusters: {(2.0, 2.0): [[0, 5], [4, -1]]}
cost: 18.00779382626432
-----------------------------
ITERATION: 2
point: [1, 4]
cluster: (2.0, 2.0)
cheapest cluster: (2.0, 2.0)
new clusters: {(1.6666666666666667, 2.6666666666666665): [[0, 5], [4, -1], [1, 4]]}
cost: 8.7042886774825
-----------------------------
ITERATION: 3
point: [0, 5]
cluster: (1.6666666666666667, 2.6666666666666665)
cheapest cluster: (1.6666666666666667, 2.6666666666666665)
